In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [5]:
B, T, C = 4, 8, 2
# (batch, time, channels) / channels is vocab_size
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [6]:
# We want x[b, t] = mean_{i <= t} x[b, i]
xbag_of_words = torch.zeros((B, T, C))
for b in range(B):
    for t in range (T):
        xprev = x[b, :t+1] # (t,C)
        xbag_of_words[b,t] = torch.mean(xprev, 0)

In [11]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print(a)

print(b)

print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [8]:
xbag_of_words[0]

tensor([[-2.0555,  1.8275],
        [-0.3760,  0.6887],
        [ 0.1984,  1.0228],
        [ 0.1177,  0.3465],
        [ 0.0888,  0.2920],
        [ 0.2493,  0.3563],
        [ 0.2575,  0.1987],
        [ 0.3182,  0.2848]])

In [15]:
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)

xbag_of_words_2 = weights @ x
print(torch.allclose(xbag_of_words, xbag_of_words_2))

print(xbag_of_words[0], xbag_of_words_2[0])

True
tensor([[-2.0555,  1.8275],
        [-0.3760,  0.6887],
        [ 0.1984,  1.0228],
        [ 0.1177,  0.3465],
        [ 0.0888,  0.2920],
        [ 0.2493,  0.3563],
        [ 0.2575,  0.1987],
        [ 0.3182,  0.2848]]) tensor([[-2.0555,  1.8275],
        [-0.3760,  0.6887],
        [ 0.1984,  1.0228],
        [ 0.1177,  0.3465],
        [ 0.0888,  0.2920],
        [ 0.2493,  0.3563],
        [ 0.2575,  0.1987],
        [ 0.3182,  0.2848]])


In [30]:
torch.manual_seed(1337)
B, T, C = 4, 8 ,32
x = torch.randn(B, T, C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

weights = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# weights = torch.zeros((T, T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)

v = value(x)
output = weights @ v

print(output.shape)
print(weights[0])

torch.Size([4, 8, 16])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)
